## 1. Import knižníc

In [ ]:
import numpy as np
np.random.seed(42)


## 2. Base `Layer` class
Each layer in the network should be able to perform:
1. **Forward** traversal (prediction)
2. **Backward** traversal (gradient computation and weight update)

Here we define a generic `Layer` class that other layers will inherit.

In [ ]:
class Layer:

    def __init__(self):
        pass

    def forward(self, inp):
     return inp

    def backward(self, inp, grad_outp):
       return grad_outp


## 3. Activation functions
We define `ReLU` and `Sigmoid` as special layers inheriting from the `Layer` class.

In [ ]:
class ReLU(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, inp):
        output = np.maximum(0,inp)
        return output

    def backward(self, inp, grad_outp):
        bool_mask = inp > 0
        return grad_outp * bool_mask



class Sigmoid(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, inp):
        output = 1/(1 + np.exp(-inp))
        return output

    def backward(self, inp, grad_outp):
        output  = self.forward(inp) * (1 - self.forward(inp)) * grad_outp
        return output


In [ ]:
class Softmax(Layer):
    def __init__(self):
        super().__init__()

    def forward(self, inp):

        shifted_inp = inp - np.max(inp, axis=1, keepdims=True)
        exps = np.exp(shifted_inp)
        output = exps / np.sum(exps, axis=1, keepdims=True)
        self.output = output  
        return output

    def backward(self, inp, grad_outp):
        
        batch_size = inp.shape[0]
        grad_inp = np.zeros_like(inp)
        
        for i in range(batch_size):
            s = self.output[i]  
            grad = grad_outp[i]  
            s = s.reshape(-1, 1)  
            jac = np.diagflat(s) - np.dot(s, s.T)  
            grad_inp[i] = np.dot(grad, jac)  
        
        return grad_inp

## 4. Dense layer
This layer has parameters (weights `W` and biases `b`) and performs the calculation of a linear transformation: \( z = xW + b \).

In [ ]:
class Dense(Layer):
    def __init__(self, inp_units, outp_units, learning_rate=0.1):
        super().__init__()
        self.lr = learning_rate
        self.outp_units = outp_units
        self.W = np.random.randn(inp_units, outp_units) * 0.1
        self.b = np.zeros(outp_units)
        
    def forward(self, inp):
        z = np.dot(inp, self.W) + self.b
        return z

    def backward(self, inp, grad_outp):

        dW = np.dot(inp.T, grad_outp)
        db = np.sum(grad_outp, axis=0)
        grad_inp = np.dot(grad_outp, self.W.T) 
        self.W -= self.lr *dW
        self.b -= self.lr * db
        return grad_inp


## 5. The MLP class itself
Contains several layers (Dense + activation), supports:
- `add_layer(...)` to add a new layer
- `forward(X)` to pass forward through the entire network
- `predict(X)` to predict
- `fit(X, y)` to train the network

In [ ]:
class MLP:
    def __init__(self):
        self.layers = []  
        self.layer_inputs = []

    def add_layer(self, neuron_count, inp_shape=None, activation='ReLU', learning_rate=0.1):
        inp_units = 0
        if inp_shape is not None:
            inp_units = inp_shape
        else:
            last_dense = None
            for layer in reversed(self.layers):
                if isinstance(layer, Dense):
                    last_dense = layer
                    break
            if last_dense is None:
                raise ValueError("input shape is not specified")
            inp_units = last_dense.outp_units
        
        dense_layer = Dense(inp_units=inp_units, outp_units=neuron_count, learning_rate=learning_rate)
        self.layers.append(dense_layer)
        
        if activation == "ReLU":
            self.layers.append(ReLU())
        elif activation == 'Sigmoid':
            self.layers.append(Sigmoid())
        elif activation == 'Softmax':
            self.layers.append(Softmax())
        elif activation is None:
            pass
        else:
            raise ValueError("Unknown activation:", activation)
        
    def forward(self, X):

        activation = X
        for layer in self.layers:
            self.layer_inputs.append(activation)
            activation = layer.forward(activation)
        return activation

    def predict(self, X):
        return self.forward(X)

    def fit(self, X, y, epochs=10):
        
        N = X.shape[0]
        for epoch in range(epochs):
            prediction = self.forward(X) 
            Loss = (1 / (2 * N)) * np.sum((y - prediction)**2) 
            
            grad_outp = (y-prediction) / N
            for i in range(len(self.layers) - 1, -1, -1):
                layer = self.layers[i]
                inp = self.layer_inputs[i]
                grad_outp = layer.backward(inp, grad_outp)
            print(f"Epoch {epoch} MSE = {Loss}")
        


## 6. Testing part (main)
After completing all TODO, it will be possible to create the network, add layers and call the `predict(...)` or `fit(...)` methods.

In [ ]:
if __name__ == "__main__":
    # Dummy test
    X = np.array([[0, 0], [1, 0], [0, 1], [2, 0], [0, 2], [1, 1]])
    y = np.array([[1, 0, 0],  
                [0, 1, 0],  
                [0, 1, 0],  
                [0, 0, 1],  
                [0, 0, 1],  
                [0, 1, 0]])
    network = MLP()
    network.add_layer(neuron_count=4, inp_shape=2, activation='ReLU')  
    network.add_layer(neuron_count=3, activation='Softmax')
    network.fit(X, y, epochs=1000)
    print("Предсказания:", network.predict(X))



## Instructions
- **Fill in** the missing implementation details in each class.
- Test (custom cases or add `network.fit(...)`).
- Extend as needed (more layers, different activation functions).

After successful completion, you should be able to create a network, train it on a small data set, and make predictions.

In [ ]:
import pandas as pd
from sklearn import datasets

In [ ]:
iris = datasets.load_iris()
df = pd.DataFrame(iris.data, columns = iris.feature_names)
df['target'] = iris.target

In [ ]:
df.head()

In [ ]:
df.sample(10)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
print(df)

In [ ]:
print(df[10:21])

In [ ]:
specific_df = df[["sepal length (cm)", 'target']]

In [ ]:
specific_df.head()

In [ ]:
df.iloc[5]


In [ ]:
df.loc[df['target'] == 1]

In [ ]:
df["target"].value_counts()

In [ ]:
sum_df = df["sepal length (cm)"].sum()
mean_df = df["sepal length (cm)"].mean()
median_df = df["sepal length (cm)"].median()

print(f"Sum: {sum_df}, \nmean: {mean_df}, \nMedian: {median_df}")

In [ ]:
min_df = df["sepal length (cm)"].min()
max_df = df["sepal length (cm)"].max()

print(f"Minimum:{min_df}, maximum: {max_df}")

In [ ]:
cols = df.columns
print(cols)

In [ ]:
cols = cols[0:4]
df1  = df[cols]
df["total values"] = df1.sum(axis=1)
df.head()

In [ ]:
newcols={
    "sepal length (cm)" : "sepallength",
    "sepal width (cm)" : "sepalwidth",
    "petal length (cm)" : "petallength",
    "petal width (cm)" : "petalwidth"
}
df.rename(columns=newcols, inplace=True)
df.columns

In [ ]:
df.style

In [35]:
df.isnull().sum()

sepallength     0
sepalwidth      0
petallength     0
petalwidth      0
target          0
total values    0
dtype: int64

In [37]:
import seaborn as sns

sns.lmplot('sepallength', 'sepalwidth', data=df,fit_reg=True)

TypeError: lmplot() got multiple values for argument 'data'